In [240]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
from numpy.random import seed
seed(1)
import requests
from tensorflow import random
random.set_seed(2)
import hvplot.pandas
%matplotlib inline

In [241]:
load_dotenv()

True

In [242]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [296]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2018-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["PYPL"]#, "SQ"]#,# "MELI"]
fintech_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
fintech_ticker.dropna(inplace=True)
fintech_ticker.head(5)
#apple_ticker.dropna(inplace=True)

PYPL                                  
                             open     high      low  close   volume
time                                                               
2018-01-02 00:00:00-05:00  74.235  74.5000  73.4300  73.83  6009642
2018-01-03 00:00:00-05:00  74.300  76.8499  74.0500  76.77  8012610
2018-01-04 00:00:00-05:00  77.330  77.9287  76.2599  76.72  7246995
2018-01-05 00:00:00-05:00  77.110  78.9300  77.0100  78.64  9758882
2018-01-08 00:00:00-05:00  78.700  79.1800  77.7730  79.04  5761316

In [317]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2014-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2017-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["PYPL"]#, "LINK"]
fintech_ticker2 = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
fintech_ticker2.dropna(inplace=True)
fintech_ticker2.tail(10)
#apple_ticker.dropna(inplace=True)

PYPL                              
                            open   high    low  close   volume
time                                                          
2017-12-15 00:00:00-05:00  74.51  75.87  73.51  75.53  7647741
2017-12-18 00:00:00-05:00  76.29  76.34  75.20  75.32  6112955
2017-12-19 00:00:00-05:00  75.39  75.39  73.91  74.49  6206748
2017-12-20 00:00:00-05:00  74.64  74.90  73.76  73.97  4819059
2017-12-21 00:00:00-05:00  74.19  74.47  73.80  74.02  3751690
2017-12-22 00:00:00-05:00  73.80  74.04  72.90  73.89  2781920
2017-12-26 00:00:00-05:00  73.29  74.34  72.63  74.28  3840751
2017-12-27 00:00:00-05:00  74.25  74.93  74.00  74.58  3362284
2017-12-28 00:00:00-05:00  74.96  75.00  73.86  74.17  2779892
2017-12-29 00:00:00-05:00  73.92  74.53  73.58  73.63  3099939

In [320]:
fintech_ticker.append(fintech_ticker2)
fintech_ticker.head()

PYPL                                  
                             open     high      low  close   volume
time                                                               
2018-01-02 00:00:00-05:00  74.235  74.5000  73.4300  73.83  6009642
2018-01-03 00:00:00-05:00  74.300  76.8499  74.0500  76.77  8012610
2018-01-04 00:00:00-05:00  77.330  77.9287  76.2599  76.72  7246995
2018-01-05 00:00:00-05:00  77.110  78.9300  77.0100  78.64  9758882
2018-01-08 00:00:00-05:00  78.700  79.1800  77.7730  79.04  5761316

In [308]:
fintech_df = pd.DataFrame()
fintech_df["PayPal"] = fintech_ticker["PYPL"]["close"]
#fintech_df["Square"] = fintech_ticker["SQ"]["close"]
#fintech_df["Affirm"] = fintech_ticker["AFRM"]["close"]
#fintech_df["Mercalibre"] = fintech_ticker["MELI"]["close"]
#fintech_df["Johnson&Johnson"] = nasdaq_ticker["JNJ"]["close"]
#apple_df = apple_df.pct_change().dropna()
fintech_df.index = fintech_df.index.date 
fintech_df.dropna(inplace=True)
fintech_df.head()

,PayPal
2018-01-02,73.83
2018-01-03,76.77
2018-01-04,76.72
2018-01-05,78.64
2018-01-08,79.04


In [325]:
fintech_df.to_csv("second_half.csv")

In [321]:
fintech_df2 = pd.DataFrame()
fintech_df2["PayPal"] = fintech_ticker2["PYPL"]["close"]
#fintech_df["Square"] = fintech_ticker["SQ"]["close"]
#fintech_df["Affirm"] = fintech_ticker["AFRM"]["close"]
#fintech_df["Mercalibre"] = fintech_ticker["MELI"]["close"]
#fintech_df["Johnson&Johnson"] = nasdaq_ticker["JNJ"]["close"]
#apple_df = apple_df.pct_change().dropna()
fintech_df2.index = fintech_df2.index.date 
fintech_df2.dropna(inplace=True)
fintech_df2.head()

,PayPal
2015-07-06,36.71
2015-07-07,36.55
2015-07-08,34.68
2015-07-09,34.49
2015-07-10,34.89


In [326]:
fintech_df2.to_csv("first_half.csv")

In [246]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [247]:
window_size = 20
feature_column = 0
target_column = 0
X, y = window_data(fintech_df, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[61.45 61.27 61.25 62.   61.13 62.64 62.56 62.01 62.24 62.51 62.91 63.82
  64.75 64.62 65.09 63.52 62.94 63.33 63.93 64.03]
 [61.27 61.25 62.   61.13 62.64 62.56 62.01 62.24 62.51 62.91 63.82 64.75
  64.62 65.09 63.52 62.94 63.33 63.93 64.03 64.17]
 [61.25 62.   61.13 62.64 62.56 62.01 62.24 62.51 62.91 63.82 64.75 64.62
  65.09 63.52 62.94 63.33 63.93 64.03 64.17 64.23]
 [62.   61.13 62.64 62.56 62.01 62.24 62.51 62.91 63.82 64.75 64.62 65.09
  63.52 62.94 63.33 63.93 64.03 64.17 64.23 64.01]
 [61.13 62.64 62.56 62.01 62.24 62.51 62.91 63.82 64.75 64.62 65.09 63.52
  62.94 63.33 63.93 64.03 64.17 64.23 64.01 65.83]] 

y sample values:
[[64.17]
 [64.23]
 [64.01]
 [65.83]
 [66.04]]


In [248]:
print(X)

[[ 61.45  61.27  61.25 ...  63.33  63.93  64.03]
 [ 61.27  61.25  62.   ...  63.93  64.03  64.17]
 [ 61.25  62.    61.13 ...  64.03  64.17  64.23]
 ...
 [303.69 308.47 306.82 ... 278.28 271.1  268.84]
 [308.47 306.82 300.48 ... 271.1  268.84 270.53]
 [306.82 300.48 301.98 ... 268.84 270.53 272.93]]


In [249]:
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]

In [250]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [251]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[1.29376567e-03]
  [5.66022479e-04]
  [4.85162125e-04]
  [3.51742541e-03]
  [0.00000000e+00]
  [2.55619573e-03]
  [2.23159945e-03]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [3.58408341e-03]
  [7.37180793e-03]
  [6.84234106e-03]
  [8.75656743e-03]
  [2.36223679e-03]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [8.18129755e-05]]

 [[5.66022479e-04]
  [4.85162125e-04]
  [3.51742541e-03]
  [0.00000000e+00]
  [6.10495674e-03]
  [2.23159945e-03]
  [0.00000000e+00]
  [9.33214315e-04]
  [1.09653576e-03]
  [1.62628070e-03]
  [3.70581528e-03]
  [7.37180793e-03]
  [6.84234106e-03]
  [8.75656743e-03]
  [2.36223679e-03]
  [0.00000000e+00]
  [1.58840060e-03]
  [2.44758097e-03]
  [4.08931054e-04]
  [6.54503804e-04]]

 [[4.85162125e-04]
  [3.51742541e-03]
  [0.00000000e+00]
  [6.10495674e-03]
  [5.78151532e-03]
  [0.00000000e+00]
  [9.33214315e-04]
  [2.02872677e-03]
  [2.72103318e-03]
  [5.32606928e-03]
  [7.49307705e-03]
  [6

In [252]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [253]:
model = Sequential()

#Adding number_units to the architecture enables the model to far more accurate that with a single digit number
number_units = 30
dropout_fraction = 0.2


# Layer 1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [254]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [255]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 20, 30)            3840      
_________________________________________________________________
dropout_27 (Dropout)         (None, 20, 30)            0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 20, 30)            7320      
_________________________________________________________________
dropout_28 (Dropout)         (None, 20, 30)            0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dropout_29 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [256]:
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=1, verbose=1)

Epoch 1/20
686/686 [==============================] - 8s 8ms/step - loss: 5.0941e-04
Epoch 2/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0019
Epoch 3/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0022
Epoch 4/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0021
Epoch 5/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0017
Epoch 6/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0011
Epoch 7/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0011
Epoch 8/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0010
Epoch 9/20
686/686 [==============================] - 6s 8ms/step - loss: 0.0010
Epoch 10/20
686/686 [==============================] - 6s 8ms/step - loss: 8.6827e-04
Epoch 11/20
686/686 [==============================] - 6s 9ms/step - loss: 7.9534e-04
Epoch 12/20
686/686 [==============================] - 6s 8ms/step - loss: 8.3517e-04
Epoch 13/

In [257]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 1s 4ms/step - loss: 0.0510


0.051020920276641846

In [258]:
predicted = model.predict(X_test)

In [259]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [260]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = fintech_df.index[-len(real_prices): ])
stocks.head(10)

,Real,Predicted
2020-06-24,168.08,165.633667
2020-06-25,172.51,165.190826
2020-06-26,170.87,165.323547
2020-06-29,168.45,165.256393
2020-06-30,174.20,164.415741
2020-07-01,177.46,165.174393
2020-07-02,177.23,167.071350
2020-07-06,176.20,168.322159
2020-07-07,177.16,168.364685
2020-07-08,181.60,168.180649


In [261]:
stocks.hvplot.line(label = "LSTM Model", width = 1000)
#(label = "Predicting Apple's Close with the S&P500 Close - (6Day Window)", xlabel="Date", ylabel="Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)